In [1]:
import os, sys
os.chdir('../')

In [2]:
import argparse
import torch
from tqdm import tqdm
import data_loader.data_loaders as module_data
import loss as module_loss
import model.metric as module_metric
import model.model as module_arch

import easydict
import torch.nn as nn
import torch.nn.functional as F
import sys
import os
import json
import random
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt 

import data_loader.data_loaders as module_data
import model.model as module_arch

from selection.svd_classifier import *
from selection.gmm import *
from selection.util import *

from utils.parse_config import ConfigParser
from utils.util import *
from utils.args import *

In [3]:
config_file = './hyperparams/multistep/config_cifar100_cce_rn34.json'
with open(config_file, 'r') as f:
    config = json.load(f)

# resume_path = './rn34/multistep_asym_40_elr.pth'

In [4]:
def decode(path):
    items = path.split('_')
    noisetype = True if items[2]=='asym' else False
    noiserate = float(items[3]) * 0.01
    
    return noisetype, noiserate, items[4].split('.')[0]

In [5]:
def make_parse(resume_path, config, noise_rate, noisetype):
    parse = easydict.EasyDict({
    "load_name" : resume_path,
    "reinit": False,
    "distill_mode": 'fine-kmeans'
    })
    
    config['trainer']['percent'] = noise_rate
    config['trainer']['asym'] = noisetype
    
    return parse, config

In [10]:
def return_statistics(dataloader, clean_labels, datanum):
    predict = np.zeros(datanum)
    for idx in clean_labels: predict[idx] = 1
        
    isNoisy_list = compute_noiseratio(dataloader)
    r_stats = []
    
    tp = (isNoisy_list[predict==1]==1).sum() # positive clean
    tn = (isNoisy_list[predict==0]==0).sum() # negative noisy
    fp = (isNoisy_list==0).sum() - tn
    fn = (isNoisy_list==1).sum() - tp
    
    print('Noisy: {}, Clean: {}'.format((isNoisy_list==0).sum(), (isNoisy_list==1).sum()))
    precision = tp / (tp + fp)
    recall = tp / (tp + fn)
    specificity = tn / (tn + fp)
    accuracy = (tp + tn) / (tp + tn + fp + fn)
    sel_samples = int(fp + tp)
    frac_clean = tp / (fp + tp)
    F1 =  tp / (tp + (fp + fn)/2)

    r_stats.extend([sel_samples, round(precision, 4), round(recall, 4), round(specificity, 4), round(accuracy, 4), round(frac_clean, 4)])
    print('Selected samples: {} \nPrecision: {} \nRecall: {} \nSpecificity: {}\nAccuracy: {} \nFraction of clean samples/selected samples: {}'.format(r_stats[0], r_stats[1], r_stats[2], r_stats[3], r_stats[4], r_stats[5]))
    
    return r_stats[0], r_stats[1], r_stats[2], r_stats[3], r_stats[4], F1

In [7]:
def extract_cleanidx(teacher, data_loader, parse, print_statistics = True):
    teacher.load_state_dict(torch.load('./checkpoint/' + parse.load_name)['state_dict'])
    teacher = teacher.cuda()

    if not parse.reinit: teacher.load_state_dict(torch.load('./checkpoint/' + parse.load_name)['state_dict'])
    for params in teacher.parameters(): params.requires_grad = False
    
    if 'fine' in parse.distill_mode:
        features, labels = get_features(teacher, data_loader)
        clean_labels = fine(current_features=features, current_labels=labels, fit = parse.distill_mode)
    elif 'loss' in parse.distill_mode:
        clean_labels, labels = cleansing_loss(teacher, data_loader)
    else:
        raise NotImplemented 
    if print_statistics: 
        selected, precision, recall, specificity, accuracy, F1 = return_statistics(data_loader, clean_labels, datanum=len(labels))
    
    return selected, precision, recall, specificity, accuracy, F1

In [8]:
def make_pd_list(root, config, log_filename):
    random.seed(config['seed'])
    torch.manual_seed(config['seed'])
    torch.cuda.manual_seed_all(config['seed'])
    torch.backends.cudnn.deterministic = True
    np.random.seed(config['seed'])
    
    # load checkpoint path
    pathlist = os.listdir(root)
    pathlist = [path for path in pathlist if 'c100' in path]
#     pathlist = [path for path in pathlist if ('.pth' in path) and ('eigen' not in path) and ('kmeans' not in path) and ('c100') not in path]
    
    # initialize model
    model = module_arch.resnet34(num_classes=100)
    
    # make pandas file
    logcolumns = ['noisetype', 'noiserate', 'lossfunction', 'selected', 'precision', 'recall', 'specificity', 'accuracy', 'F1']
    log_pd = pd.DataFrame(np.zeros([len(pathlist), len(logcolumns)]), columns = logcolumns)
    
    # write pandas file
    noisetypelst = ['']
    for i in range(len(pathlist)):
        noisetype, noiserate, lossfunction = decode(pathlist[i])
        parse, config = make_parse('./rn34/' + pathlist[i], config, noiserate, noisetype)
        
        # load original dataloader
        data_loader = getattr(module_data, config['data_loader']['type'])(
        config['data_loader']['args']['data_dir'],
        batch_size= 100,
        shuffle=False,
        validation_split=0.0,
        num_batches=config['data_loader']['args']['num_batches'],
        training=True,
        num_workers=config['data_loader']['args']['num_workers'],
        pin_memory=config['data_loader']['args']['pin_memory'],
        config=config)
        
        selected, precision, recall, specificity, accuracy, F1 = extract_cleanidx(model, data_loader, parse)
        log_pd.loc[i] = [str(noisetype), str(noiserate), lossfunction, selected, precision, recall, specificity, accuracy, F1]
        log_pd.to_csv(log_filename)
        
    return log_pd

In [ ]:
make_pd_list(root = './checkpoint/rn34/', config=config, log_filename = 'c100_n_fine_pretrained_statistics.csv')

Files already downloaded and verified
Train: 50000 Val: 0


100%|██████████| 500/500 [00:05<00:00, 105.49it/s]


Noisy: 9993, Clean: 40007
Selected samples: 37441 
Precision: 0.9972 
Recall: 0.9333 
Specificity: 0.9896
Accuracy: 0.9445 
Fraction of clean samples/selected samples: 0.9972
Files already downloaded and verified
Train: 50000 Val: 0


100%|██████████| 500/500 [00:04<00:00, 117.35it/s]


Noisy: 9993, Clean: 40007
Selected samples: 37174 
Precision: 0.9838 
Recall: 0.9141 
Specificity: 0.9396
Accuracy: 0.9192 
Fraction of clean samples/selected samples: 0.9838
Files already downloaded and verified
Train: 50000 Val: 0


...

100%|██████████| 500/500 [00:05<00:00, 116.42it/s]


Noisy: 29703, Clean: 20297
Selected samples: 22034 
Precision: 0.8561 
Recall: 0.9293 
Specificity: 0.8932
Accuracy: 0.9079 
Fraction of clean samples/selected samples: 0.8561
Files already downloaded and verified
Train: 50000 Val: 0


100%|██████████| 500/500 [00:04<00:00, 115.17it/s]


Noisy: 4969, Clean: 45031
Selected samples: 42859 
Precision: 0.9957 
Recall: 0.9477 
Specificity: 0.9632
Accuracy: 0.9492 
Fraction of clean samples/selected samples: 0.9957
Files already downloaded and verified
Train: 50000 Val: 0


 49%|████▊     | 243/500 [00:20<00:23, 11.04it/s]